In [ ]:
import requests as req
import pandas as pd
import numpy as np
import json
import mysql.connector
import time
import random

token = "RGAPI-a765fb0c-23d2-478f-a0bd-d1ad87d32a47"

In [ ]:
oraclesdb = pd.read_excel("oraclesdata/2019-spring-match-data-OraclesElixir-2019-05-21.xlsx").groupby("url").first()
oraclesdb = oraclesdb[["gameid", "league", "split", "date", "week", "game", "patchno"]]

In [ ]:
def convertUrls(url, prefix="https://acs.leagueoflegends.com/v1/stats/game/"):
    if type(url) is str:
        u = url.split("/")
        game = "/".join(u[len(u)-2:])
        timeline = game.split("?")
        return {"game_url" : prefix+game, "timeline_url" : prefix+timeline[0]+"/timeline?"+timeline[1]}
        
acs = pd.DataFrame({url : convertUrls(url) for url in oraclesdb.index}).T

In [ ]:
games = oraclesdb.merge(acs, left_index=True, right_index=True)

In [ ]:
games.head(3)

In [ ]:
def buildQuery(table, kv, mode="i"):
    r = []
    if "c" in mode:
        r.append("CREATE TABLE `{}` (\n{}) ENGINE=InnoDB DEFAULT CHARSET=utf8;".format(
        table,
        ",\n".join(["  `{}` {} DEFAULT NULL".format(k, "int(11)" if isinstance(v, int) else "varchar(255)") for k,v in kv.items()])))
    if "i" in mode:
        r.append("INSERT INTO {} ({}) VALUES ({});".format(
        table, 
        ", ".join(kv.keys()), 
        ", ".join(["'{}'".format(v) for k,v in kv.items()])))
    return r

def queryFromUrls( game, mode="i" ):
    g = json.loads(req.get(game["game_url"]).text)
    q = []
    q += buildQuery("GameMetadata", {
        "gameId": g["gameId"],
        "platformId": g["platformId"],
        "league": game["league"],
        "split": game["split"],
        "week": game["week"],
        "nogame": game["game"]
    }, mode)
    q += buildQuery("Game", {
        "gameId": g["gameId"],
        "platformId": g["platformId"],
        "mode": g["gameMode"],
        "type": g["gameType"],
        "duration": g["gameDuration"],
        "created": g["gameCreation"],
        "version": g["gameVersion"],
        "patch": ".".join(g["gameVersion"].split(".")[0:2]),
        "seasonId": g["seasonId"],
        "mapId": g["mapId"],
        "queueId": g["queueId"]
    }, mode)
    for t in g["teams"]:
        q += buildQuery("Team", {
            "gameId": g["gameId"],
            "platformId": g["platformId"],
            "side": {100:"BLU", 200:"RED"}[t["teamId"]],
            "win": {"Win":1, "Fail":0}[t["win"]],
            "firstBlood": int(t["firstBlood"]),
            "firstTower": int(t["firstTower"]),
            "firstInhibitor": int(t["firstInhibitor"]),
            "firstBaron": int(t["firstBaron"]),
            "firstDragon": int(t["firstDragon"]),
            "firstRiftHerald": int(t["firstRiftHerald"]),
            "towerKills": t["towerKills"],
            "inhibitorKills": t["inhibitorKills"],
            "baronKills": t["baronKills"],
            "dragonKills": t["dragonKills"],
            "riftHeraldKills": t["riftHeraldKills"]
        }, mode)
        for b in t["bans"]:
            q += buildQuery("Ban", {
                "gameId": g["gameId"],
                "platformId": g["platformId"],
                "side": {100:"BLU", 200:"RED"}[t["teamId"]],
                "championId": b["championId"],
                "pickTurn": b["pickTurn"]
            }, mode)
    names = {pi["participantId"]:pi["player"]["summonerName"] for pi in g["participantIdentities"]}
    for p in g["participants"]:
        s = p["stats"]
        q += buildQuery("Player", {
            "gameId": g["gameId"],
            "platformId": g["platformId"],
            "side": {100:"BLU", 200:"RED"}[p["teamId"]],
            "rawname": names[p["participantId"]],
            "team": names[p["participantId"]].split(" ")[0],
            "name": " ".join(names[p["participantId"]].split(" ")[1:]),
            "championId": p["championId"],
            "summonerSpell1": p["spell1Id"],
            "summonerSpell2": p["spell2Id"],
            "participantId" : s["participantId"],
            "win" : int(s["win"]),
            "item0" : s["item0"],
            "item1" : s["item1"],
            "item2" : s["item2"],
            "item3" : s["item3"],
            "item4" : s["item4"],
            "item5" : s["item5"],
            "item6" : s["item6"],
            "kills" : s["kills"],
            "deaths" : s["deaths"],
            "assists" : s["assists"],
            "largestKillingSpree" : s["largestKillingSpree"],
            "largestMultiKill" : s["largestMultiKill"],
            "killingSprees" : s["killingSprees"],
            "longestTimeSpentLiving" : s["longestTimeSpentLiving"],
            "doubleKills" : s["doubleKills"],
            "tripleKills" : s["tripleKills"],
            "quadraKills" : s["quadraKills"],
            "pentaKills" : s["pentaKills"],
            "unrealKills" : s["unrealKills"],
            "totalDamageDealt" : s["totalDamageDealt"],
            "magicDamageDealt" : s["magicDamageDealt"],
            "physicalDamageDealt" : s["physicalDamageDealt"],
            "trueDamageDealt" : s["trueDamageDealt"],
            "largestCriticalStrike" : s["largestCriticalStrike"],
            "totalDamageDealtToChampions" : s["totalDamageDealtToChampions"],
            "magicDamageDealtToChampions" : s["magicDamageDealtToChampions"],
            "physicalDamageDealtToChampions" : s["physicalDamageDealtToChampions"],
            "trueDamageDealtToChampions" : s["trueDamageDealtToChampions"],
            "totalHeal" : s["totalHeal"],
            "totalUnitsHealed" : s["totalUnitsHealed"],
            "damageSelfMitigated" : s["damageSelfMitigated"],
            "damageDealtToObjectives" : s["damageDealtToObjectives"],
            "damageDealtToTurrets" : s["damageDealtToTurrets"],
            "visionScore" : s["visionScore"],
            "timeCCingOthers" : s["timeCCingOthers"],
            "totalDamageTaken" : s["totalDamageTaken"],
            "magicalDamageTaken" : s["magicalDamageTaken"],
            "physicalDamageTaken" : s["physicalDamageTaken"],
            "trueDamageTaken" : s["trueDamageTaken"],
            "goldEarned" : s["goldEarned"],
            "goldSpent" : s["goldSpent"],
            "turretKills" : s["turretKills"],
            "inhibitorKills" : s["inhibitorKills"],
            "totalMinionsKilled" : s["totalMinionsKilled"],
            "neutralMinionsKilled" : s["neutralMinionsKilled"],
            "neutralMinionsKilledTeamJungle" : s["neutralMinionsKilledTeamJungle"],
            "neutralMinionsKilledEnemyJungle" : s["neutralMinionsKilledEnemyJungle"],
            "totalTimeCrowdControlDealt" : s["totalTimeCrowdControlDealt"],
            "champLevel" : s["champLevel"],
            "visionWardsBoughtInGame" : s["visionWardsBoughtInGame"],
            "sightWardsBoughtInGame" : s["sightWardsBoughtInGame"],
            "wardsPlaced" : s["wardsPlaced"],
            "wardsKilled" : s["wardsKilled"],
            "firstBloodKill" : int(s["firstBloodKill"]),
            "firstBloodAssist" : int(s["firstBloodAssist"]),
            "firstTowerKill" : int(s["firstTowerKill"]),
            "firstTowerAssist" : int(s["firstTowerAssist"]),
            "firstInhibitorKill" : int(s["firstInhibitorKill"]),
            "firstInhibitorAssist" : int(s["firstInhibitorAssist"]),
            "combatPlayerScore" : s["combatPlayerScore"],
            "objectivePlayerScore" : s["objectivePlayerScore"],
            "totalPlayerScore" : s["totalPlayerScore"],
            "totalScoreRank" : s["totalScoreRank"],
            "playerScore0" : s["playerScore0"],
            "playerScore1" : s["playerScore1"],
            "playerScore2" : s["playerScore2"],
            "playerScore3" : s["playerScore3"],
            "playerScore4" : s["playerScore4"],
            "playerScore5" : s["playerScore5"],
            "playerScore6" : s["playerScore6"],
            "playerScore7" : s["playerScore7"],
            "playerScore8" : s["playerScore8"],
            "playerScore9" : s["playerScore9"],
            "perk0" : s["perk0"],
            "perk0Var1" : s["perk0Var1"],
            "perk0Var2" : s["perk0Var2"],
            "perk0Var3" : s["perk0Var3"],
            "perk1" : s["perk1"],
            "perk1Var1" : s["perk1Var1"],
            "perk1Var2" : s["perk1Var2"],
            "perk1Var3" : s["perk1Var3"],
            "perk2" : s["perk2"],
            "perk2Var1" : s["perk2Var1"],
            "perk2Var2" : s["perk2Var2"],
            "perk2Var3" : s["perk2Var3"],
            "perk3" : s["perk3"],
            "perk3Var1" : s["perk3Var1"],
            "perk3Var2" : s["perk3Var2"],
            "perk3Var3" : s["perk3Var3"],
            "perk4" : s["perk4"],
            "perk4Var1" : s["perk4Var1"],
            "perk4Var2" : s["perk4Var2"],
            "perk4Var3" : s["perk4Var3"],
            "perk5" : s["perk5"],
            "perk5Var1" : s["perk5Var1"],
            "perk5Var2" : s["perk5Var2"],
            "perk5Var3" : s["perk5Var3"],
            "perkPrimaryStyle" : s["perkPrimaryStyle"],
            "perkSubStyle" : s["perkSubStyle"],
            "statPerk0" : s["statPerk0"],
            "statPerk1" : s["statPerk1"],
            "statPerk2" : s["statPerk2"],
            "role": p["timeline"]["role"]+"-"+p["timeline"]["lane"]
        }, mode)
    return q

In [ ]:
#generate linkage dicts
#print(",\n".join(['"{0}" : s["{0}"]'.format(k) for k in g["participants"][0]["stats"].keys()]))

In [ ]:
with open('../safe_connector.json') as _:
    dbc = json.load(_)
cnx = mysql.connector.connect(**dbc)
cur = cnx.cursor()
i=0
for index, row in games.iterrows():
    try:
        for q in queryFromUrls(row):
            cur.execute(q)
        print("{} : {}".format(i, index))
        cnx.commit()
    except Exception as err:
        print("{}\n\n{}\n{}\n\n{}".format(err, index, row, q))
        cnx.rollback()
    i+=1
    time.sleep(random.randrange(10,20,step=1)/10)
cur.close()
cnx.close()